# Project: Data Wrangling with Twitter data

## Table of Contents
<ul>    
<li><a href="#intro">Introduction</a></li>
<li><a href="#gather">Gather</a></li>
<li><a href="#assess">Assess</a></li>
<li><a href="#clean">Clean</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
<li><a href="#ref">References</a></li>
</ul>

<a id='intro'></a>
## Introduction

In this project I'm going to analyze the dataset from twitter account WeRateDogs®
<br>
using Tweepy to query Twitter's API for additional data: retweet count and favorite count
<br>
Assessing data
Cleaning data
Storing, analyzing, and visualizing your wrangled data
Reporting on 1) your data wrangling efforts and 2) your data analyses and visualizations

<a id='gather'></a>
## Gather

In [1]:
#Import libraries
import pandas as pd
import requests 
import os
import tweepy
import json
import numpy as np

#### Archive table

In [3]:
df_archive = pd.read_csv("twitter-archive-enhanced.csv")
df_archive.head(1)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None


In [4]:
df_archive.shape

(2356, 17)

#### Image predictions table

In [5]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)

In [6]:
with open(os.path.join(os.getcwd(), url.split('/')[-1]), mode='wb') as file:
          file.write(response.content)

In [7]:
df_predictions = pd.read_csv('image-predictions.tsv', sep='\t')
df_predictions.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


#### Tweepy
create an API object to gather Twitter data.

In [8]:
consumer_key = '7GCntbM7icOGMHkkXjcQXfTkL'
consumer_secret = 'gZP0QgAihs5EoDZFi6PdfwkDfill046cWS1fRZajz84mgVgpxB'
access_token = '960852542-Q9H69Zz43N7xvQEAEY25il9Xl5P3ZAjVnfzc2HEe'
access_secret = 'xM4iTrao32Su1Ww2ygacFoZtfTBGpzGz0u5uEZLmqsMcl'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [ ]:
#get data from Twitter
id_list = df.tweet_id.astype(str)#[0:10]
tweets = []
error_count = 0
error_ids = []
for tweet_id in id_list:
    try:
        tweet = api.get_status(tweet_id, tweet_mode='extended')
        tweets.append(tweet._json)
    except tweepy.TweepError as e:
        print(e)
        error_ids.append(tweet_id)
error_ids

In [ ]:
#Write json data to file
with open('tweet_json.txt', 'w') as file:
    json.dump(tweets, file)

In [10]:
#Read json data from file
ls_tweets = []
with open('tweet_json.txt') as file:
    data = json.load(file)
    for p in data:
        ls_tweets.append({'tweet_id': p['id'],
                        'retweet_count': p['retweet_count'],
                        'favorite_count': p['favorite_count']})
        

    

In [11]:
len(ls_tweets)

2332

In [12]:
# create dataFrame from list 
df_tweets = pd.DataFrame(ls_tweets, columns = ['tweet_id', 'retweet_count', 'favorite_count'])
df_tweets.head()

,tweet_id,retweet_count,favorite_count
0,892420643555336193,7675,36055
1,892177421306343426,5674,31097
2,891815181378084864,3763,23407
3,891689557279858688,7850,39325
4,891327558926688256,8445,37558


## Assess Data

#### Archive table

In [14]:
df_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [16]:
#Are columns in_reply_to_status_id, in_reply_to_user_id are the same records
df_archive.query('in_reply_to_status_id == in_reply_to_status_id and in_reply_to_user_id ==in_reply_to_user_id', engine='python').shape

(78, 17)

In [17]:
#Are columns retweeted_status_id, retweeted_status_user_id, retweeted_status_timestamp
#are the same records
df_archive.query('retweeted_status_id ==retweeted_status_id and retweeted_status_user_id ==retweeted_status_user_id').shape

(181, 17)

#### Issues
##### df_ archive table
1) Original records have these columns equal NaN<br>
- in_reply_to_status_id<br>
- in_reply_to_user_id<br>
- retweeted_status_id<br>
- retweeted_status_user_id<br>
- retweeted_status_timestamp<br>

2) Columns to delete: *timestamp, source, expanded_urls* <br>
3) *rating_denominator* has some incorrect data, zeros, big numbers (decimal?)<br>
4) *rating_numerator* can be decimal like 13.5/10 tweet_id:883482846933004288<br>
  Some records have *rating_numerator* = 0 or >20<br>
5) *name* columns have some errors like name 'None' or 'a'. I'm not sure it will be used for analysis<br>
6) *doggo, floofer, pupper, puppo* columns have values only in 380 records vs 433 in *text* column<br>
7) Tidiness: *doggo, floofer, pupper, puppo* can be combined in one column<br>

In [18]:
#Checking for duplicated data
df_archive[df_archive.duplicated()].shape

(0, 17)

In [19]:
df_archive['rating_denominator'].describe()

count    2356.000000
mean       10.455433
std         6.745237
min         0.000000
25%        10.000000
50%        10.000000
75%        10.000000
max       170.000000
Name: rating_denominator, dtype: float64

In [22]:
# strange rating_denominator values
df_archive[df_archive['rating_denominator'] !=10]['rating_denominator'].unique()

array([  0,  15,  70,   7,  11, 150, 170,  20,  50,  90,  80,  40, 130,
       110,  16, 120,   2])

In [23]:
#name column
df_archive['name'].describe()

count     2356
unique     957
top       None
freq       745
Name: name, dtype: object

In [25]:
#df_archive['name'].value_counts()

In [30]:
# *doggo, floofer, pupper, puppo* columns
df_archive.query("doggo != 'None' or floofer != 'None' or pupper != 'None' or puppo != 'None'").shape

(380, 17)

In [27]:
print('Puppo:' + str(df_archive[df_archive['text'].str.contains("[Pp]uppo")].shape))
print('Doggo:' + str(df_archive[df_archive['text'].str.contains("[Dd]oggo")].shape))
print('Floofer:' + str(df_archive[df_archive['text'].str.contains("[Ff]loofer")].shape))
print('Pupper:' + str(df_archive[df_archive['text'].str.contains("[Pp]upper")].shape))

Puppo:(38, 17)
Doggo:(106, 17)
Floofer:(10, 17)
Pupper:(279, 17)


#### Image prediction table

In [ ]:
df_predictions.head(2)

In [ ]:
df_predictions.info()

#### Issues
##### df_predictions table

- *jpg_url, img_num* columns are not needed <br>
- p1, p2, p3* some breeds start with capital letter, some not<br>
- Names of columns p1,p2,p3, p1_conf,p2_conf,p3_conf, p1_dog,p2_dog,p3_dog re not obvious<br>
- Some first chose predictions are not dogs<br>
- For analysis only one most probable prediction required
- Some tweets do not have dog breed prediction, all p_dog flags are false

In [31]:
df_predictions.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


In [32]:
#df_predictions.p1.value_counts()
#df_predictions.p2.value_counts()
df_predictions.p3.value_counts()

Labrador_retriever                79
Chihuahua                         58
golden_retriever                  48
Eskimo_dog                        38
kelpie                            35
kuvasz                            34
Staffordshire_bullterrier         32
chow                              32
beagle                            31
cocker_spaniel                    31
toy_poodle                        29
Pomeranian                        29
Pekinese                          29
Pembroke                          27
Great_Pyrenees                    27
Chesapeake_Bay_retriever          27
malamute                          26
French_bulldog                    26
American_Staffordshire_terrier    24
Cardigan                          23
pug                               23
basenji                           21
bull_mastiff                      20
toy_terrier                       20
Siberian_husky                    19
Boston_bull                       17
Shetland_sheepdog                 17
b

In [33]:
#Tweets in archive table and not in prediction table
len(list(set(df_archive.tweet_id) - set(df_predictions.tweet_id)))

281

In [36]:
#Are there errors in confidence intervals?
print(len(df_predictions[df_predictions.p1_conf < df_predictions.p2_conf]))
print(len(df_predictions[df_predictions.p1_conf < df_predictions.p3_conf]))

0
0


In [37]:
#Some first chose predictions are not dogs
len(df_predictions[df_predictions.p1_dog==False])

543

In [41]:
len(df_predictions[((df_predictions.p1_dog==False)&(df_predictions.p2_dog==False)&(df_predictions.p3_dog==False))])

324

#### Tweepy table

In [42]:
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2332 entries, 0 to 2331
Data columns (total 3 columns):
tweet_id          2332 non-null int64
retweet_count     2332 non-null int64
favorite_count    2332 non-null int64
dtypes: int64(3)
memory usage: 54.7 KB


In [43]:
df_tweets.retweet_count.describe()

count     2332.000000
mean      2692.102058
std       4553.026319
min          1.000000
25%        544.750000
50%       1260.500000
75%       3128.000000
max      77473.000000
Name: retweet_count, dtype: float64

In [44]:
df_tweets.favorite_count.describe()

count      2332.000000
mean       7525.440395
std       11685.867567
min           0.000000
25%        1311.250000
50%        3273.500000
75%        9218.250000
max      155529.000000
Name: favorite_count, dtype: float64

#### Issues
##### df_tweets table
- Merge df_tweets and df_archive table. df_tweets is just additional info about the same tweets <br>
- Some tweets were deleted, df_tweets has no info about them, ids are in error_ids list

## Clean

##### df_archive_copy table
1) Columns to delete: timestamp, source, expanded_urls<br>
2) Delete replies and retweets<br>
Delete records with not null data in these columns:
- in_reply_to_status_id
- in_reply_to_user_id
- retweeted_status_id
- retweeted_status_user_id
- retweeted_status_timestamp

Drop those columns <br>

3) Clean *rating_denominator* column
- Set *rating_denominator* = 10 where rating_denominator !=10 but text column has correct rating
- Records with *rating_denominator* !=10 are not ratings of one dog, drop these records, drop *rating_denominator* column

4) Clean *rating_numerator* column
- Change type of rating_numerator as decimal
- Extract rating_numerator from tweet text

5)Clean and combine *doggo, floofer, pupper, puppo* columns
- Extract dog stage from tweet text for *doggo, floofer, pupper, puppo* columns
- Convert all dog stages name to lowercase
- Combine *doggo, floofer, pupper, puppo* in one dog_stage column 
- Drop columns *doggo, floofer, pupper, puppo, text, name*

##### df_predictions_copy table
1) Columns to delete: *jpg_url,img_num*<br>
2) Rename columns p1,p2,p3, p1_conf,p2_conf,p3_conf, p1_dog,p2_dog,p3_dog<br>
3) Convert dog breeds to lowercase<br>
4) **Tidiness:** Choose only one prediction with dog flag = True, drop other predictions<br>
5) Drop all records where dog breed is not predicted<br>

##### Merge all three tables into df_all_tweets
**Tidiness:**<br> 
Some tweets were deleted from twitter and some archive records were cleaned as invalid.<br>
Predictions data for which dog breed was identified.<br>
Only records presented in all three tables will go to the merged table


In [118]:
#Create copies of the tables
df_archive_copy = df_archive.copy()
df_predictions_copy = df_predictions.copy()
df_tweets_copy = df_tweets.copy()

##### df_archive_copy table
Columns to delete: *timestamp, source, expanded_urls* <br>

In [119]:
#Drop columns
df_archive_copy.drop(['timestamp', 'source', 'expanded_urls'], axis=1, inplace=True)
#Test
df_archive_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 14 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), object(7)
memory usage: 257.8+ KB


##### Delete replies and retweets
Delete not null data in these columns:<br>
- in_reply_to_status_id<br>
- in_reply_to_user_id<br>
- retweeted_status_id<br>
- retweeted_status_user_id<br>
- retweeted_status_timestamp<br>

Drop those columns <br>

In [120]:
#drop replies
df_archive_copy.drop(index = df_archive_copy[df_archive_copy.in_reply_to_status_id.isnull()==False].index, inplace=True)

In [121]:
#drop retweets 
df_archive_copy.drop(index = df_archive_copy[df_archive_copy.retweeted_status_id.isnull()==False].index, inplace=True)

In [122]:
#Test
print(df_archive_copy[df_archive_copy.in_reply_to_status_id.isnull()==False].shape)
print(df_archive_copy[df_archive_copy.in_reply_to_user_id.isnull()==False].shape)
print(df_archive_copy[df_archive_copy.retweeted_status_user_id.isnull()==False].shape)
print(df_archive_copy[df_archive_copy.retweeted_status_id.isnull()==False].shape)
print(df_archive_copy[df_archive_copy.retweeted_status_timestamp.isnull()==False].shape)
#all should be zero

(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)


In [123]:
df_archive_copy.drop(['in_reply_to_status_id', 'in_reply_to_user_id','retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp'], axis=1, inplace=True)

In [124]:
df_archive_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2355
Data columns (total 9 columns):
tweet_id              2097 non-null int64
text                  2097 non-null object
rating_numerator      2097 non-null int64
rating_denominator    2097 non-null int64
name                  2097 non-null object
doggo                 2097 non-null object
floofer               2097 non-null object
pupper                2097 non-null object
puppo                 2097 non-null object
dtypes: int64(3), object(6)
memory usage: 163.8+ KB


##### Clean *rating_denominator* column
- Set *rating_denominator* = 10 where rating_denominator !=10 but text column has correct rating
- records with *rating_denominator* !=10 are not ratings of one dog, drop these recors, drop *rating_denominator* column<br>

In [125]:
#rating_denominator !=10 but text column has correct rating
df_archive_copy[(df_archive_copy.rating_denominator !=10) & (df_archive_copy.text.str.contains(r"/10"))].shape

(5, 9)

In [126]:
#Number of problematic records
df_archive_copy[df_archive_copy.rating_denominator !=10].shape

(17, 9)

In [127]:
#Set rating_denominator = 10 
#where rating_denominator !=10 but text column has correct rating
df_archive_copy.loc[(df_archive_copy.rating_denominator !=10) & (df_archive_copy.text.str.contains(r"/10")), ['rating_denominator']] = 10

In [128]:
len(df_archive_copy[df_archive_copy.rating_denominator !=10])

12

In [129]:
#drop records which do not have rating of one dog
#denominator=7 -> no rating
#denominator>10-> multiple dogs
df_archive_copy.drop(index = df_archive_copy[df_archive_copy.rating_denominator !=10].index, inplace=True)

In [130]:
#Test: check all recors for rating_denominator and rating number inside text column
print(df_archive_copy[(df_archive_copy.rating_denominator ==10) & (df_archive_copy.text.str.contains(r"/10"))].shape)
print(df_archive_copy.shape)
#should be the same

(2085, 9)
(2085, 9)


In [131]:
# rating_denominator = 10 in all records, so we can drop the column
df_archive_copy.drop(['rating_denominator'], axis=1, inplace=True)

##### Clean *rating_numerator* column
- Change type of rating_numerator as decimal
- Extract rating_numerator from tweet text

In [132]:
df_archive_copy.rating_numerator.astype(str).unique()

array(['13', '12', '14', '5', '11', '6', '10', '0', '75', '27', '3', '7',
       '8', '9', '4', '1776', '50', '26', '2', '1', '420'], dtype=object)

In [133]:
#Extract rating_numerator from tweet text and compare with column
#Change type of rating_numerator as decimal
df_archive_copy['num'] = df_archive_copy.text.str.extract('(\d+\.?\d*)/10').astype(float)
df_archive_copy['rating_numerator'] = df_archive_copy.rating_numerator.astype(float)

In [134]:
df_archive_copy.query('rating_numerator != num')[['tweet_id','text', 'rating_numerator', 'num']]

,tweet_id,text,rating_numerator,num
45,883482846933004288,This is Bella. She hopes her smile made you sm...,5.0,13.50
695,786709082849828864,"This is Logan, the Chow who lived. He solemnly...",75.0,9.75
763,778027034220126208,This is Sophie. She's a Jubilant Bush Pupper. ...,27.0,11.27
1068,740373189193256964,"After so many requests, this is Bretagne. She ...",9.0,14.00
1165,722974582966214656,Happy 4/20 from the squad! 13/10 for all https...,4.0,13.00
1202,716439118184652801,This is Bluebert. He just saw that both #Final...,50.0,11.00
1662,682962037429899265,This is Darrel. He just robbed a 7/11 and is i...,7.0,10.00
1712,680494726643068929,Here we have uncovered an entire battalion of ...,26.0,11.26
2335,666287406224695296,This is an Albanian 3 1/2 legged Episcopalian...,1.0,9.00


In [135]:
#column num has correct values, so I just update rating_numerator with num
df_archive_copy['rating_numerator'] = df_archive_copy['num']
df_archive_copy.drop(['num'], axis=1, inplace=True)

In [136]:
#Leave to check rating_numerator extreme values to after merging with predictions data
df_archive_copy[df_archive_copy['rating_numerator'] > 25]

,tweet_id,text,rating_numerator,name,doggo,floofer,pupper,puppo
979,749981277374128128,This is Atticus. He's quite simply America af....,1776.0,Atticus,None,None,None,None
2074,670842764863651840,After so many requests... here you go.\n\nGood...,420.0,None,None,None,None,None


##### Clean *doggo, floofer, pupper, puppo* columns
- Extract dog stage from tweet text for *doggo, floofer, pupper, puppo* columns
- Convert all dog stages name to lowercase
- Combine *doggo, floofer, pupper, puppo* in one dog_stage column 
- Drop columns *doggo, floofer, pupper, puppo, text, name*

In [137]:
#Not null dog stage data
df_archive_copy.query("doggo != 'None' or floofer != 'None' or pupper != 'None' or puppo != 'None'").shape

(336, 8)

In [138]:
#Extract dog stage from tweet text
df_archive_copy['pupper'] = df_archive_copy.text.str.extract('([Pp]upper)');
df_archive_copy['doggo'] = df_archive_copy.text.str.extract('([Dd]oggo)');
df_archive_copy['floofer'] = df_archive_copy.text.str.extract('([Ff]loofer)');
df_archive_copy['puppo'] = df_archive_copy.text.str.extract('([Pp]uppo)');

In [139]:
#Combine dog stages in one column 
df_archive_copy.doggo.replace(np.nan, '', regex=True, inplace=True)
df_archive_copy.puppo.replace(np.nan, '', regex=True, inplace=True)
df_archive_copy.floofer.replace(np.nan, '', regex=True, inplace=True)
df_archive_copy.pupper.replace(np.nan, '', regex=True, inplace=True)

In [140]:
#Convert all dog stages name to lowercase
df_archive_copy['pupper'] = df_archive_copy.pupper.str.lower()
df_archive_copy['doggo'] = df_archive_copy.doggo.str.lower()
df_archive_copy['floofer'] = df_archive_copy.floofer.str.lower()
df_archive_copy['puppo'] = df_archive_copy.puppo.str.lower()

In [141]:
df_archive_copy['dog_stage'] = df_archive_copy['doggo']+ df_archive_copy['puppo'] + df_archive_copy['floofer'] + df_archive_copy['pupper'] 

In [142]:
df_archive_copy.dog_stage.value_counts()

                1721
pupper           238
doggo             77
puppo             28
floofer            9
doggopupper        9
doggopuppo         2
doggofloofer       1
Name: dog_stage, dtype: int64

In [143]:
# multiple stages: all puppers are doggos, but not all doggos are puppers
df_archive_copy.loc[df_archive_copy.dog_stage == 'doggopupper', 'dog_stage'] = 'pupper'
df_archive_copy.loc[df_archive_copy.dog_stage == 'doggopuppo', 'dog_stage'] = 'puppo'
df_archive_copy.loc[df_archive_copy.dog_stage == 'doggofloofer', 'dog_stage'] = 'floofer'

In [144]:
df_archive_copy.dog_stage.value_counts()

           1721
pupper      247
doggo        77
puppo        30
floofer      10
Name: dog_stage, dtype: int64

In [145]:
#drop columns we don't need any more
df_archive_copy.drop(['pupper','doggo','puppo','floofer', 'text', 'name'], axis=1, inplace=True)

In [146]:
df_archive_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2085 entries, 0 to 2355
Data columns (total 3 columns):
tweet_id            2085 non-null int64
rating_numerator    2085 non-null float64
dog_stage           2085 non-null object
dtypes: float64(1), int64(1), object(1)
memory usage: 65.2+ KB


##### df_predictions table
- Columns to delete: *jpg_url, img_num*<br>
- Rename columns p1,p2,p3, p1_conf,p2_conf,p3_conf, p1_dog,p2_dog,p3_dog<br>
- Convert dog breeds to lowercase<br>
- Choose only one most probable prediction with dog flag = True, drop other predictions<br>
- Drop all records where dog breed is not predicted<br>

In [147]:
#Drop jpg_url, img_num columns
df_predictions_copy.drop(['jpg_url', 'img_num'], axis=1, inplace=True)
#Test
df_predictions_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 10 columns):
tweet_id    2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(1), object(3)
memory usage: 119.6+ KB


In [149]:
#rename columns
df_predictions_copy = df_predictions_copy.rename(columns={'p1': 'dog_breed', 'p1_conf': 'conf_int', 'p1_dog': 'is_dog', 'p2': 'dog_breed2', 'p2_conf': 'conf_int2', 'p2_dog': 'is_dog2', 'p3': 'dog_breed3', 'p3_conf': 'conf_int3', 'p3_dog': 'is_dog3'})

In [150]:
#Convert dog breeds to lowercase
df_predictions_copy['dog_breed'] = df_predictions_copy.dog_breed.str.lower()
df_predictions_copy['dog_breed2'] = df_predictions_copy.dog_breed2.str.lower()
df_predictions_copy['dog_breed3'] = df_predictions_copy.dog_breed3.str.lower()

In [151]:
#Test
df_predictions_copy.head()

,tweet_id,dog_breed,conf_int,is_dog,dog_breed2,conf_int2,is_dog2,dog_breed3,conf_int3,is_dog3
0,666020888022790149,welsh_springer_spaniel,0.465074,True,collie,0.156665,True,shetland_sheepdog,0.061428,True
1,666029285002620928,redbone,0.506826,True,miniature_pinscher,0.074192,True,rhodesian_ridgeback,0.072010,True
2,666033412701032449,german_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,miniature_pinscher,0.560311,True,rottweiler,0.243682,True,doberman,0.154629,True


In [152]:
#Number non dogs
len(df_predictions_copy[df_predictions_copy['is_dog'] == False])

543

In [153]:
#if first prediction isn't dog, use second or third if exists
def choose_dog(row):
    if (row['is_dog'] == False & row['is_dog2']):
        row['dog_breed'] = row['dog_breed2']
        row['conf_int'] = row['conf_int2']
        row['is_dog'] = row['is_dog2']
    if (row['is_dog'] == False & row['is_dog3']):
        row['dog_breed'] = row['dog_breed3']
        row['conf_int'] = row['conf_int3']
        row['is_dog'] = row['is_dog3']
    return row['is_dog'], row['conf_int'], row['is_dog'] 

df_predictions_copy[['is_dog','conf_int','is_dog']] = df_predictions_copy.apply(choose_dog, axis=1, result_type="expand")

In [154]:
#Number non dogs
len(df_predictions_copy[df_predictions_copy['is_dog'] == False])

324

In [155]:
#Drop all records where dog breed is not predicted
df_predictions_copy.drop(index=df_predictions_copy[df_predictions_copy['is_dog'] == False].index, inplace=True)

In [156]:
#Test Number non dogs
len(df_predictions_copy[df_predictions_copy['is_dog'] == False])

0

In [157]:
#drop second and third predictions as less probable
#is_dog column has the same value True for all the records and can be deleted
df_predictions_copy.drop(['dog_breed2','conf_int2','is_dog2','dog_breed3', 'conf_int3', 'is_dog3', 'is_dog'], axis=1, inplace=True)

In [158]:
#Test
df_predictions_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1751 entries, 0 to 2073
Data columns (total 4 columns):
tweet_id     1751 non-null int64
dog_breed    1751 non-null object
conf_int     1751 non-null float64
is_dog       1751 non-null bool
dtypes: bool(1), float64(1), int64(1), object(1)
memory usage: 56.4+ KB


##### Merge all three tables into df_all_tweets
Some tweets were deleted from twitter and some archive records were cleaned as invalid.<br>
Predictions data for which dog breed was identified.<br>
Only records presented in all three tables will go to the merged table

In [159]:
#inner join for tweepy and archive data
df_all_tweets = pd.merge(df_archive_copy, df_tweets_copy, on='tweet_id', how='inner')

In [160]:
#Test
df_all_tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2078 entries, 0 to 2077
Data columns (total 5 columns):
tweet_id            2078 non-null int64
rating_numerator    2078 non-null float64
dog_stage           2078 non-null object
retweet_count       2078 non-null int64
favorite_count      2078 non-null int64
dtypes: float64(1), int64(3), object(1)
memory usage: 97.4+ KB


In [161]:
#inner join for tweepy and archive data and breed prediction data
df_all_tweets = pd.merge(df_all_tweets, df_predictions_copy, on='tweet_id', how='inner')

In [162]:
df_all_tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1648 entries, 0 to 1647
Data columns (total 8 columns):
tweet_id            1648 non-null int64
rating_numerator    1648 non-null float64
dog_stage           1648 non-null object
retweet_count       1648 non-null int64
favorite_count      1648 non-null int64
dog_breed           1648 non-null object
conf_int            1648 non-null float64
is_dog              1648 non-null bool
dtypes: bool(1), float64(2), int64(3), object(2)
memory usage: 104.6+ KB


In [166]:
#Check on extreme rating_numerator values
df_all_tweets[df_all_tweets['rating_numerator'] >25]
#They were not dogs with identified breed 

,tweet_id,rating_numerator,dog_stage,retweet_count,favorite_count,dog_breed,conf_int,is_dog


### Storing, Analyzing, and Visualizing Data for this Project
Store the clean DataFrame(s) in a CSV file with the main one named twitter_archive_master.csv. If additional files exist because multiple tables are required for tidiness, name these files appropriately.

In [ ]:
At least three (3) insights and one (1) visualization must be produced.

<a id='ref'></a>
## References

https://stackoverflow.com/questions/28384588/twitter-api-get-tweets-with-specific-id
<br>
https://stackabuse.com/reading-and-writing-json-to-a-file-in-python/
<br>
https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object<br>
https://stackoverflow.com/questions/37863660/pandas-dataframe-query-fetch-not-null-rows-pandas-equivalent-to-sql-is-no<br>
https://docs.python.org/3/library/re.html<br>
